# Week 2 Homework — Architectures & TCP Sockets

Instructions:
- Complete all tasks. Use Python 3.9+ and localhost.
- Provide code, outputs, and short analysis.
- You may reuse helpers from the examples notebook, but explain any changes.

Rubric: correctness, framing robustness, handling of edge cases, clear analysis.

---

## Task 1 — Length-prefixed echo server and client
Implement an echo server and client using 4-byte big-endian length-prefixed framing.

Requirements:
- Server handles multiple clients (threads or `selectors`)
- Client sends 100 messages and verifies echoes
- Include graceful shutdown


In [ ]:
import socket, struct, threading, time
from contextlib import closing


def send_msg(sock: socket.socket, data: bytes):
    sock.sendall(struct.pack('!I', len(data)) + data)


def recv_all(sock: socket.socket, n: int) -> bytes:
    buf = bytearray()
    while len(buf) < n:
        chunk = sock.recv(n - len(buf))
        if not chunk:
            raise ConnectionError('socket closed')
        buf.extend(chunk)
    return bytes(buf)


def recv_msg(sock: socket.socket) -> bytes:
    (length,) = struct.unpack('!I', recv_all(sock, 4))
    return recv_all(sock, length)


class EchoServer:
    def __init__(self, host='127.0.0.1', port=50666):
        self.addr = (host, port)
        self.sock = None
        self.running = threading.Event()

    def start(self):
        self.running.set()
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind(self.addr)
        self.sock.listen(64)
        threading.Thread(target=self._accept_loop, daemon=True).start()

    def _accept_loop(self):
        while self.running.is_set():
            try:
                c, _ = self.sock.accept()
            except OSError:
                break
            threading.Thread(target=self._client_loop, args=(c,), daemon=True).start()

    def _client_loop(self, c: socket.socket):
        try:
            while self.running.is_set():
                data = recv_msg(c)
                send_msg(c, data)
        except Exception:
            pass
        finally:
            try: c.close()
            except Exception: pass

    def stop(self):
        self.running.clear()
        try: self.sock.close()
        except Exception: pass


server = EchoServer(); server.start()


### Client validation
Send 100 messages and assert responses match.


In [ ]:
from contextlib import closing

def client_check(n=100, addr=('127.0.0.1', 50666)):
    with closing(socket.create_connection(addr, timeout=3)) as s:
        for i in range(n):
            msg = f"msg-{i}".encode()
            send_msg(s, msg)
            echo = recv_msg(s)
            assert echo == msg
    print('Client OK for', n, 'messages')

client_check(100)
server.stop()


## Task 2 — Robust framing (partial reads and coalescing)
Explain (markdown) how your framing avoids issues when TCP delivers partial segments or merges multiple messages. Provide a short code snippet that demonstrates reading two messages back-to-back without mis-parsing.


### Explanation
Write 5–8 sentences here describing length-prefix framing and how `recv_all` handles partial reads and sticky packets.


## Task 3 — Basic metrics
Instrument your server to print messages/sec and average service time every 2s. Provide a short load (5 concurrent clients, 200 messages each) and include sample metrics output.


In [ ]:
import threading

def spam_client(count=200, addr=('127.0.0.1', 50666)):
    with closing(socket.create_connection(addr, timeout=3)) as s:
        for i in range(count):
            send_msg(s, f"hello-{i}".encode())
            _ = recv_msg(s)

class MeasuredEchoServer(EchoServer):
    def __init__(self, host='127.0.0.1', port=50666):
        super().__init__(host, port)
        self.msg_count = 0
        self.total_dt = 0.0

    def _client_loop(self, c: socket.socket):
        import time
        try:
            while self.running.is_set():
                t0 = time.perf_counter()
                data = recv_msg(c)
                send_msg(c, data)
                dt = time.perf_counter() - t0
                self.msg_count += 1
                self.total_dt += dt
        except Exception:
            pass
        finally:
            try: c.close()
            except Exception: pass

    def start(self):
        super().start()
        threading.Thread(target=self._metrics_loop, daemon=True).start()

    def _metrics_loop(self):
        import time
        last = time.time(); last_count = 0
        while self.running.is_set():
            time.sleep(2)
            now = time.time(); elapsed = now - last
            count = self.msg_count; delta = count - last_count
            rps = delta/elapsed if elapsed>0 else 0
            avg_ms = (self.total_dt/count)*1e3 if count else 0
            print({"total": count, "rps": round(rps,2), "avg_ms": round(avg_ms,3)})
            last = now; last_count = count

server = MeasuredEchoServer(); server.start()

threads = [threading.Thread(target=spam_client, daemon=True) for _ in range(5)]
for t in threads: t.start()
for t in threads: t.join()

time.sleep(3)
server.stop()


### Analysis (Task 3)
Paste a few lines of metrics and explain what they show about throughput and latency under this light load.
